<a href="https://colab.research.google.com/github/Likithareddy99/Missing_lab_tests/blob/main/Automatic_Diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.4 MB/s 
     |████████████████████████████████| 365 kB 65.2 MB/s 
     |████████████████████████████████| 6.6 MB 32.9 MB/s 
     |████████████████████████████████| 120 kB 53.3 MB/s 
     |████████████████████████████████| 212 kB 48.4 MB/s 
     |████████████████████████████████| 115 kB 63.1 MB/s 
     |████████████████████████████████| 127 kB 69.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-diagnosis-prediction")
model = AutoModelForSequenceClassification.from_pretrained("bvanaken/CORe-clinical-diagnosis-prediction")

In [ ]:
# input = '''PRESENT ILLNESS: 58yo man w/ hx of hypertension, AFib on coumadin presented to ED with the worst headache of his life. Brother reports states that patient has been complaining of headache for 2 days and that the patient has lost consciousness. He had a syncopal episode and was intubated by EMS, MEDICATION ON ADMISSION: 1mg IV ativan x 1, metformin, 
# PHYSICAL EXAM: Vitals: physical examination: 90 passive hemagglutination test: 21 electron microscope: 91 BP: 151/78. Cardiac: RRR. GCS  E: 3   V:2  M:5 HEENT: atraumatic, normocephalic Pupils: 4-3mm. Abd: Soft, BS+ Extrem: Warm and well-perfused'''

input = '''PRESENT ILLNESS: 40yo man w/ vomiting, blister, rash, fatigue, scar, fever, abdominal pain, headache, pimple. Brother reports states that patient has been complaining of headache for 2 days and that the patient has lost consciousness. He had a syncopal episode and was intubated by EMS, 
MEDICATION ON ADMISSION: 1mg metisazone, tecovirimat x 1
PHYSICAL EXAM: Vitals: electron microscope: 91 BP: 151/78. Cardiac: RRR. GCS  E: 3   V:2  M:5 HEENT: atraumatic, normocephalic Pupils: 4-3mm. Abd: Soft, BS+ Extrem: Warm and well-perfused'''

tokenized_input = tokenizer(input, return_tensors="pt")
output = model(**tokenized_input)

import torch
predictions = torch.sigmoid(output.logits)
predicted_labels = [model.config.id2label[_id] for _id in (predictions > 0.3).nonzero()[:, 1].tolist()] 

In [ ]:
predicted_labels

['427',
 '482',
 '790',
 'acute',
 'adverse',
 'due',
 'effects',
 'elsewhere',
 'failure',
 'organism',
 'pneumonia',
 'staphylococcus',
 'susceptible',
 'therapeutic',
 'unspecified',
 'use']

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
tokenizer_ner = AutoTokenizer.from_pretrained("samrawal/bert-base-uncased_clinical-ner")
model_ner = AutoModelForTokenClassification.from_pretrained("samrawal/bert-base-uncased_clinical-ner")

In [ ]:
from transformers import pipeline
Bert_ner = pipeline('ner',model=model_ner,tokenizer=tokenizer_ner,grouped_entities=True,ignore_subwords=True)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:136: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="first"` instead.
  "`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to"
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:141: UserWarning: `ignore_subwords` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="first"` instead.
  "`ignore_subwords` is deprecated and will be removed in version v5.0.0, defaulted to"


In [ ]:
result = Bert_ner(input)

In [ ]:
result

[{'entity_group': 'problem',
  'score': 0.9877949,
  'word': 'vomiting',
  'start': 29,
  'end': 37},
 {'entity_group': 'problem',
  'score': 0.9979406,
  'word': 'blister',
  'start': 39,
  'end': 46},
 {'entity_group': 'problem',
  'score': 0.9971437,
  'word': 'rash',
  'start': 48,
  'end': 52},
 {'entity_group': 'problem',
  'score': 0.9975942,
  'word': 'fatigue',
  'start': 54,
  'end': 61},
 {'entity_group': 'problem',
  'score': 0.99711823,
  'word': 'scar',
  'start': 63,
  'end': 67},
 {'entity_group': 'problem',
  'score': 0.9977507,
  'word': 'fever',
  'start': 69,
  'end': 74},
 {'entity_group': 'problem',
  'score': 0.9975648,
  'word': 'abdominal pain',
  'start': 76,
  'end': 90},
 {'entity_group': 'problem',
  'score': 0.99775654,
  'word': 'headache',
  'start': 92,
  'end': 100},
 {'entity_group': 'problem',
  'score': 0.9831424,
  'word': 'pimple',
  'start': 102,
  'end': 108},
 {'entity_group': 'problem',
  'score': 0.99832803,
  'word': 'headache',
  'start': 1

In [ ]:
import pandas as pd
entities=[]
for j in result:
  entities.append([j['entity_group'],j['word']])

Entity_table = pd.DataFrame(entities,columns=['entity_type','entity'])  
Entity_table

Entity_table = Entity_table.drop_duplicates()
Entity_table.reset_index(drop=True, inplace=True)
Entity_table.reset_index()

,index,entity_type,entity
0,0,problem,vomiting
1,1,problem,blister
2,2,problem,rash
3,3,problem,fatigue
4,4,problem,scar
5,5,problem,fever
6,6,problem,abdominal pain
7,7,problem,headache
8,8,problem,pimple
9,9,problem,a syncopal episode


In [ ]:
tests_performed = []
for i in range(len(Entity_table['entity_type'])):
  if Entity_table['entity_type'][i] == 'test':
    tests_performed.append(Entity_table['entity'][i])

tests_performed

['electron microscope', 'bp']